In [1]:
# Import the required packages
import os
from sklearn import preprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
original_df = pd.read_csv(r'C:\Users\luqqa\OneDrive\Escritorio\DiploDatos\Practico AS\train.csv')

In [3]:

def clasificacionDias(dia):
    if dia in ['Monday','Tuesday','Wednesday','Thursday']:
        return 'low'
    else:
        return'weekend'

In [4]:
def transform_data(train_data_fname, test_data_fname):
    df_train = pd.read_csv(train_data_fname)
    df_train['is_train_set'] = 1
    df_test = pd.read_csv(test_data_fname)
    df_test['is_train_set'] = 0

    # we  get the TripType for the train set. To do that, we group by VisitNumber and
    # then we get the max (or min or avg)
    y = df_train.groupby(["VisitNumber", "Weekday"], as_index=False).max().TripType

    # we remove the TripType now, and concat training and testing data
    # the concat is done so that we have the same columns for both datasets
    # after one-hot encoding
    df_train = df_train.drop("TripType", axis=1)
    df = pd.concat([df_train, df_test])
    
    # the next three operations are the ones we have just presented in the previous lines
    

    # drop the columns we won't use (it may be good to use them somehow)
    #df = df.drop(["Upc", "FinelineNumber"], axis=1)
    df.loc[ df.Upc.isna(), 'Upc'] = 0
    df.loc[df.FinelineNumber.isna(), 'FinelineNumber'] = 4520
    
    
    # one-hot encoding for the DepartmentDescription
    df = pd.get_dummies(df, columns=["DepartmentDescription"], dummy_na=True)
    
    df = df.drop(['DepartmentDescription_nan'], axis=1)

    # now we add the groupby values
    df = df.groupby(["VisitNumber", "Weekday"], as_index=False).sum()
    
    #now we apply a function to do a segmentation of the weekdays             
    df['Segmentacion Semana']=df.Weekday.apply(lambda x:clasificacionDias(x))
    df = pd.get_dummies(df, columns=["Segmentacion Semana"], dummy_na=True)

    df = df.drop(['Segmentacion Semana_nan'], axis=1)
    
    # finally, we do one-hot encoding for the Weekday
    df = pd.get_dummies(df, columns=["Weekday"], dummy_na=True)
    
    df = df.drop(["Weekday_nan"], axis=1)

    # get train and test back
    df_train = df[df.is_train_set != 0]
    df_test = df[df.is_train_set == 0]
    
    X = df_train.drop(["is_train_set"], axis=1)
    yy = None
    XX = df_test.drop(["is_train_set"], axis=1)

    return X, y, XX, yy

In [5]:
X, y, XX, yy = transform_data(r'C:\Users\luqqa\OneDrive\Escritorio\DiploDatos\Practico AS\train.csv',r'C:\Users\luqqa\OneDrive\Escritorio\DiploDatos\Practico AS\test.csv')

In [6]:
X.shape

(67029, 81)

In [7]:
XX.describe()

,VisitNumber,Upc,ScanCount,FinelineNumber,DepartmentDescription_1-HR PHOTO,DepartmentDescription_ACCESSORIES,DepartmentDescription_AUTOMOTIVE,DepartmentDescription_BAKERY,DepartmentDescription_BATH AND SHOWER,DepartmentDescription_BEAUTY,...,DepartmentDescription_WIRELESS,Segmentacion Semana_low,Segmentacion Semana_weekend,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
count,28645.000000,2.864500e+04,28645.000000,28645.000000,28645.000000,28645.000000,28645.000000,28645.000000,28645.000000,28645.000000,...,28645.000000,28645.000000,28645.000000,28645.000000,28645.000000,28645.000000,28645.000000,28645.000000,28645.000000,28645.000000
mean,96016.140897,2.032540e+11,7.500785,25221.880014,0.004224,0.013161,0.056834,0.073975,0.050096,0.152487,...,0.019096,0.489614,0.510386,0.159434,0.125257,0.175458,0.175493,0.121522,0.120195,0.122639
std,55174.990728,4.230518e+11,10.191538,34505.226596,0.093887,0.159864,0.373293,0.348468,0.428489,0.626586,...,0.192238,0.499901,0.499901,0.366087,0.331016,0.380365,0.380395,0.326739,0.325196,0.328028
min,7.000000,0.000000e+00,-26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47678.000000,1.904013e+10,2.000000,4804.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,96388.000000,8.845792e+10,4.000000,12729.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,143842.000000,1.946637e+11,9.000000,30807.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,191346.000000,9.146880e+12,133.000000,437259.000000,5.000000,8.000000,14.000000,13.000000,21.000000,15.000000,...,6.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
XX

,VisitNumber,Upc,ScanCount,FinelineNumber,DepartmentDescription_1-HR PHOTO,DepartmentDescription_ACCESSORIES,DepartmentDescription_AUTOMOTIVE,DepartmentDescription_BAKERY,DepartmentDescription_BATH AND SHOWER,DepartmentDescription_BEAUTY,...,DepartmentDescription_WIRELESS,Segmentacion Semana_low,Segmentacion Semana_weekend,Weekday_Friday,Weekday_Monday,Weekday_Saturday,Weekday_Sunday,Weekday_Thursday,Weekday_Tuesday,Weekday_Wednesday
1,7,6.794963e+10,2,13435.0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
2,8,4.259239e+11,28,63189.0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
7,15,1.019672e+12,9,7880.0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
9,19,1.733264e+12,9,38013.0,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
11,23,6.922496e+11,2,8506.0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95665,191331,7.046246e+10,1,9189.0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
95666,191335,2.141193e+11,9,18519.0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
95668,191342,5.070081e+09,2,215.0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
95671,191345,1.191780e+11,17,49902.0,0,0,0,0,0,1,...,0,0,1,0,0,0,1,0,0,0


In [9]:
# split training dataset into train and "validation" 
# (we won't be using validation set in this example, because of the cross-validation;
# but it could be useful for you depending on your approach)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.27,random_state=42)

In [10]:
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn import ensemble 


In [11]:
param_grid = {
    
    'criterion': ['gini', 'entropy'],
    'random_state'      : [2],
    'max_features': ['auto'],
    "n_estimators": [100,120, 160]
    
}

In [12]:
from sklearn.model_selection import GridSearchCV



RFT = GridSearchCV(ensemble.RandomForestClassifier(),param_grid, cv=3, scoring='accuracy')
RFT.fit(X_train, y_train);
best_tree_RFT = RFT.best_estimator_
best_tree_RFT

RandomForestClassifier(n_estimators=160, random_state=2)

In [13]:
# results dataframe is used to store the computed results
results = pd.DataFrame(columns=('clf', 'best_acc'))

In [14]:
print('Best Decision Tree accuracy: ', RFT.best_score_)
print(RFT)
results = results.append({'clf': RFT, 'best_acc': RFT.best_score_}, ignore_index=True)

print('The best classifier so far is: ')
print(results.loc[results['best_acc'].idxmax()]['clf'])

Best Decision Tree accuracy:  0.6891336372637203
GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto'],
                         'n_estimators': [100, 120, 160], 'random_state': [2]},
             scoring='accuracy')
The best classifier so far is: 
GridSearchCV(cv=3, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto'],
                         'n_estimators': [100, 120, 160], 'random_state': [2]},
             scoring='accuracy')


In [15]:
yy = RFT.predict(XX)

In [16]:
submission = pd.DataFrame(list(zip(XX.VisitNumber, yy)), columns=["VisitNumber", "TripType"])

In [17]:
submission.to_csv(r"C:\Users\luqqa\OneDrive\Escritorio\DiploDatos\Practico AS\submissionRandomForestClassifier7.csv", header=True, index=False)